In [1]:
# !pip install cloud-tpu-client==0.10 torch==1.13.0 https://storage.googleapis.com/tpu-pytorch/wheels/colab/torch_xla-1.13-cp38-cp38-linux_x86_64.whl

In [2]:
import numpy as np
from tqdm import tqdm

## PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torch.optim as optim

## Torchvision
import torchvision
from torchvision.datasets import CIFAR10
from torchvision import transforms

In [4]:
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
# device = xm.xla_device()
print("Device:", device)

Device: cuda:0


In [5]:
# transformers definition
test_transform = transforms.Compose([transforms.ToTensor(),
                                     transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
                                     ])
train_transform = transforms.Compose([transforms.RandomHorizontalFlip(),
                                      transforms.RandomResizedCrop((32,32),scale=(0.8,1.0),ratio=(0.9,1.1)),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
                                     ])

# CIFAR10 dataset loading

train_set = CIFAR10(root="./data", train=True, transform=train_transform, download=True)
test_set = CIFAR10(root="./data", train=False, transform=test_transform, download=True)

# Dataloaders creation
batch_size = 200
train_loader = torch.utils.data.DataLoader(
    train_set, batch_size=batch_size, shuffle=True, num_workers=4
)
test_loader = torch.utils.data.DataLoader(
    test_set, batch_size=batch_size, shuffle=False, num_workers=4
)

# classes
classes = (
    "plane",
    "car",
    "bird",
    "cat",
    "deer",
    "dog",
    "frog",
    "horse",
    "ship",
    "truck",
)

Files already downloaded and verified
Files already downloaded and verified


In [6]:
def img_to_patch(x, patch_size, flatten_channels=True):
    """
    x is a torch.Tensor representing an image with dimensions [Batch size, Number of channels, Height, Width]. 
    The patch_size is the number of pixels per dimension of the patches 
    flatten_channels is a boolean indicating whether the patches should be returned 
    as a flattened feature vector or as an image grid."
    """
    B, C, H, W = x.shape
    x = x.reshape(B, C, H//patch_size, patch_size, W//patch_size, patch_size)
    # [B, H', W', C, p_H, p_W] then [B, H'*W', C, p_H, p_W]
    x = x.permute(0, 2, 4, 1, 3, 5).flatten(1,2)            
    if flatten_channels:
        # [B, H'*W', C*p_H*p_W]
        x = x.flatten(2,4)          
    return x

In [7]:
class AttentionBlock(nn.Module):

    def __init__(self, embed_dim, hidden_dim, num_heads, dropout=0.0):
        """
        Inputs:
            The embed_dim is the dimensionality of the input and attention feature vectors. 
            The hidden_dim is the dimensionality of the hidden layer in the feed-forward network, 
            The num_heads is the number of heads used in the Multi-Head Attention block,
            the dropout is the amount of dropout applied in the feed-forward network
        """
        super().__init__()

        self.layer_norm_1 = nn.LayerNorm(embed_dim)
        self.attn = nn.MultiheadAttention(embed_dim, num_heads,
                                          dropout=dropout)
        self.layer_norm_2 = nn.LayerNorm(embed_dim)
        self.linear = nn.Sequential(
            nn.Linear(embed_dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, embed_dim),
            nn.Dropout(dropout)
        )


    def forward(self, x):
        inp_x = self.layer_norm_1(x)
        x = x + self.attn(inp_x, inp_x, inp_x)[0]
        x = x + self.linear(self.layer_norm_2(x))
        return x

In [8]:
class VisionTransformer(nn.Module):

    def __init__(self, embed_dim, hidden_dim, num_channels, num_heads, num_layers, num_classes, patch_size, num_patches, dropout=0.0):
        """
        Inputs:
            embed_dim - Dimensionality of the input feature vectors to the Transformer
            hidden_dim - Dimensionality of the hidden layer in the feed-forward networks
                         within the Transformer
            num_channels - Number of channels of the input (3 for RGB)
            num_heads - Number of heads to use in the Multi-Head Attention block
            num_layers - Number of layers to use in the Transformer
            num_classes - Number of classes to predict
            patch_size - Number of pixels that the patches have per dimension
            num_patches - Maximum number of patches an image can have
            dropout - Amount of dropout to apply in the feed-forward network and
                      on the input encoding
        """
        super().__init__()

        self.patch_size = patch_size

        # define layers and nets
        self.input_layer = nn.Linear(num_channels*(patch_size**2), embed_dim)
        self.transformer = nn.Sequential(*[AttentionBlock(embed_dim, hidden_dim, num_heads, dropout=dropout) for _ in range(num_layers)])
        self.mlp_head = nn.Sequential(
            nn.LayerNorm(embed_dim),
            nn.Linear(embed_dim, num_classes)
        )
        self.dropout = nn.Dropout(dropout)

        # Parameters/Embeddings
        self.cls_token = nn.Parameter(torch.randn(1,1,embed_dim))
        self.pos_embedding = nn.Parameter(torch.randn(1,1+num_patches,embed_dim))


    def forward(self, x):
        # input preprocessing
        x = img_to_patch(x, self.patch_size)
        B, T, _ = x.shape
        x = self.input_layer(x)

        # CLS token and positional encoding are added
        cls_token = self.cls_token.repeat(B, 1, 1)
        x = torch.cat([cls_token, x], dim=1)
        x = x + self.pos_embedding[:,:T+1]

        # transformer application
        x = self.dropout(x)
        x = x.transpose(0, 1)
        x = self.transformer(x)

        # classification prediction
        cls = x[0]
        out = self.mlp_head(cls)
        return out

In [9]:
#define parameters
#in this case, the patch size, or length of the input sequences fed into the Transformer model, 
#was tested at values of 2, 4, and 8. A smaller patch size allows the Transformer to model more 
#complex functions but also requires more computation time due to its quadratic memory usage in the attention layer,
#and can make the task more difficult by requiring the model to learn which patches are nearby and which are 
#far away. The best performance was achieved with a patch size of 4. The embedding and hidden dimensionality
#of the Transformer were chosen to be 256 and 512, respectively, with the query-key sizes in the multi-head 
#attention layers being considered to ensure that the attention maps were not too restricted.
#dropout value of 0.2 was used to prevent overfitting.
model_kwargs={'embed_dim': 256,
'hidden_dim': 512,
'num_heads': 8,
'num_layers': 6,
'patch_size': 4,
'num_channels': 3,
'num_patches': 64,
'num_classes': 10,
'dropout': 0.2}

In [10]:
model = VisionTransformer(**model_kwargs).to(device)

In [11]:
from tqdm import tqdm

In [12]:
# AdamW optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[100,150], gamma=0.1)
# save loss
criterion = nn.CrossEntropyLoss().to(device)

# Training loop
for epoch in range(200):
    running_loss = 0.0
    for i, (images, labels) in tqdm(enumerate(train_loader)):
        # Clear the gradients
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass
        loss.backward()
        optimizer.step()

        # Print statistics
        running_loss += loss.item()
        if i % 50 == 49:  # Print every 50 mini-batches
            print("[%d, %5d] loss: %.3f" % (epoch + 1, i + 1, running_loss / 50))
            running_loss = 0.0
    scheduler.step()

print("Finished Training")

52it [00:03, 23.21it/s]

[1,    50] loss: 2.063


103it [00:05, 23.42it/s]

[1,   100] loss: 1.853


154it [00:08, 23.48it/s]

...

[200,   150] loss: 0.042


202it [00:08, 23.41it/s]

[200,   200] loss: 0.038


250it [00:10, 23.03it/s]

[200,   250] loss: 0.032
Finished Training


In [13]:
with torch.no_grad():
    correct = 0
    total = 0
    for (img, label) in tqdm(test_loader):
        img = img.to(device)
        label = label.to(device)
        outputs = model(img)
        predicted = torch.argmax(outputs.data, 1)
        total += label.size(0)
        correct += (predicted == label).sum().item()

100%|██████████| 50/50 [00:01<00:00, 47.95it/s]


In [14]:
correct / total

0.7589

In [16]:
torch.save(model.state_dict(), 'model.pt')

In [17]:
with torch.no_grad():
    correct = 0
    total = 0
    for (img, label) in tqdm(train_loader):
        img = img.to(device)
        label = label.to(device)
        outputs = model(img)
        predicted = torch.argmax(outputs.data, 1)
        total += label.size(0)
        correct += (predicted == label).sum().item()

100%|██████████| 250/250 [00:06<00:00, 41.64it/s]


In [18]:
correct / total

0.98846